In [1]:
"""
Window.ipyb

This notebook performs a trailing window on specified time series data with the following parameters:
- window size
- hops
In our context, our data is collected on an average of 1 second per movement. Thus, setting our window
 frame as 1s, with a sampling rate of 50Hz would result in 50 samples per window.
"""

import pandas as pd
import tensorflow as tf
import numpy as np
import xarray

# with open ('training_data/combined_buddhaClap.csv', 'r') as r:
csv1 = 'training_data/combined_buddhaClap.csv'
csv2 = 'training_data/combined_crankLeft.csv'
csv3 = 'training_data/combined_crankRight.csv'
csv4 = 'training_data/combined_knobLeft.csv'
csv5 = 'training_data/combined_knobRight.csv'
csv6= 'training_data/combined_pushback.csv'
csv7 = 'training_data/combined_swipe.csv'

df = pd.read_csv('training_data/test.csv')
df

,gyrox,gyroy,gyroz,acelx,acely,acelz
0,-1.75,0.19,-0.33,22.77,249.99,249.99
1,-1.65,-0.01,-0.11,90.10,249.99,245.52
2,-1.34,0.04,0.13,126.38,249.99,222.18
3,-1.06,-0.01,0.26,130.48,249.99,187.18
4,-0.83,-0.14,0.30,147.04,245.03,161.22
...,...,...,...,...,...,...
95,-1.74,1.51,-1.49,-111.26,-250.00,-169.74
96,-1.85,1.83,-1.81,-152.55,-250.00,-197.15
97,-1.64,1.74,-2.00,-127.42,-250.00,-164.90
98,-1.38,1.80,-2.18,-73.12,-250.00,-130.55


In [2]:
len(df)
df.to_numpy()

array([[-1.7500e+00,  1.9000e-01, -3.3000e-01,  2.2770e+01,  2.4999e+02,
         2.4999e+02],
       [-1.6500e+00, -1.0000e-02, -1.1000e-01,  9.0100e+01,  2.4999e+02,
         2.4552e+02],
       [-1.3400e+00,  4.0000e-02,  1.3000e-01,  1.2638e+02,  2.4999e+02,
         2.2218e+02],
       [-1.0600e+00, -1.0000e-02,  2.6000e-01,  1.3048e+02,  2.4999e+02,
         1.8718e+02],
       [-8.3000e-01, -1.4000e-01,  3.0000e-01,  1.4704e+02,  2.4503e+02,
         1.6122e+02],
       [-6.2000e-01, -2.7000e-01,  3.3000e-01,  1.8921e+02,  2.3178e+02,
         1.3690e+02],
       [-5.4000e-01, -1.9000e-01,  4.0000e-01,  2.4999e+02,  2.2418e+02,
         1.1510e+02],
       [-2.7000e-01, -2.0000e-02,  6.3000e-01,  2.4999e+02,  2.1301e+02,
         1.0772e+02],
       [ 3.0000e-02, -8.0000e-02,  9.9000e-01,  2.4999e+02,  2.1532e+02,
         1.0066e+02],
       [ 1.8000e-01,  5.0000e-02,  1.3900e+00,  2.4999e+02,  2.0455e+02,
         6.7020e+01],
       [ 1.4000e-01,  5.0000e-01,  1.7300e+00,  2.

In [5]:
"""
Sliding window function and params
"""
from tqdm import tqdm

csv1 = 'training_data/combined_buddhaClap.csv'
csv2 = 'training_data/combined_crankLeft.csv'
csv3 = 'training_data/combined_crankRight.csv'
csv4 = 'training_data/combined_knobLeft.csv'
csv5 = 'training_data/combined_knobRight.csv'
csv6= 'training_data/combined_pushback.csv'
csv7 = 'training_data/combined_swipe.csv'

window = 50
overlap = 25

from skimage.util.shape import view_as_windows
import warnings

def strided_axis0(a, L, overlap=1):
    if L==overlap:
        raise Exception("Overlap arg must be smaller than length of windows")
    S = L - overlap
    nd0 = ((len(a)-L)//S)+1
    if nd0*S-S!=len(a)-L:
        warnings.warn("Not all elements were covered")
    output = view_as_windows(a, (L,a.shape[1]), step=S)[:,0,:,:]
    print(output.shape)
    return output


# total_array = np.empty((0,6))
total_array = np.empty((0,50,6))
csv_list = [csv1, csv2, csv3, csv4, csv5, csv6, csv7]
print('start!')
for csv in tqdm(csv_list):
    df = pd.read_csv(csv)
    window_array = strided_axis0(df.to_numpy(), 50, overlap=25)
    # total_array += window_df.to_numpy()
    # window_array = window_df.to_xarray().to_array()
    total_array = np.append(total_array, window_array , axis = 0)
print('ok!')
total_array

100%|██████████| 7/7 [00:02<00:00,  3.28it/s]


start!
(10911, 50, 6)
(10911, 50, 6)
(10911, 50, 6)
(10911, 50, 6)
(10911, 50, 6)
(10911, 50, 6)
(10911, 50, 6)
ok!


array([[[-1.7500e+00,  1.9000e-01, -3.3000e-01,  2.2770e+01,
          2.4999e+02,  2.4999e+02],
        [-1.6500e+00, -1.0000e-02, -1.1000e-01,  9.0100e+01,
          2.4999e+02,  2.4552e+02],
        [-1.3400e+00,  4.0000e-02,  1.3000e-01,  1.2638e+02,
          2.4999e+02,  2.2218e+02],
        ...,
        [-7.9000e-01,  2.4900e+00, -3.6100e+00, -9.9390e+01,
          5.4380e+01,  2.3390e+01],
        [-1.1200e+00,  2.1900e+00, -3.4100e+00, -4.0690e+01,
          1.4500e+02,  9.3510e+01],
        [-1.6200e+00,  2.2000e+00, -3.0000e+00,  1.0530e+01,
          2.2304e+02,  1.2435e+02]],

       [[-2.3000e-01,  2.5000e-01,  2.4000e-01, -1.3695e+02,
         -2.0723e+02, -8.3020e+01],
        [-3.9000e-01,  1.4000e-01,  1.1000e-01, -1.1826e+02,
         -2.0531e+02, -9.0680e+01],
        [-5.0000e-01,  8.0000e-02,  5.0000e-02, -8.7340e+01,
         -2.1457e+02, -9.5150e+01],
        ...,
        [-1.8200e+00,  1.4000e-01,  1.2800e+00,  1.1925e+02,
         -8.8540e+01,  1.2853e+02],
  

In [6]:
total_array.shape

(76377, 50, 6)

In [7]:
""""
SAVE 3D (76377, 50, 6) ARRAY TO CSV
"""
filename = 'window.csv'
arr = total_array
arrReshaped = arr.reshape(arr.shape[0], -1)
# saving reshaped array to file.
np.savetxt(filename, arrReshaped)
# retrieving data from file.
loadedArr = np.loadtxt(filename)
# This loadedArr is a 2D array, therefore we need to convert it to the original array shape.
# reshaping to get original matrice with original shape.
loadedOriginal = loadedArr.reshape(loadedArr.shape[0], loadedArr.shape[1] // arr.shape[2], arr.shape[2])

In [8]:
"""
Save into txt file
"""
import numpy as np
arr = total_array
  
# reshaping the array from 3D 
# matrice to 2D matrice. 
arr_reshaped = arr.reshape(arr.shape[0], -1) 
  
# saving reshaped array to file. 
np.savetxt("numpy-window.txt", arr_reshaped) 
  
# retrieving data from file. 
loaded_arr = np.loadtxt("numpy-window.txt") 
  
# This loadedArr is a 2D array, therefore 
# we need to convert it to the original 
# array shape.reshaping to get original 
# matrice with original shape. 
load_original_arr = loaded_arr.reshape( 
    loaded_arr.shape[0], loaded_arr.shape[1] // arr.shape[2], arr.shape[2]) 
  
# check the shapes: 
print("shape of arr: ", arr.shape) 
print("shape of load_original_arr: ", load_original_arr.shape) 
  
# check if both arrays are same or not: 
if (load_original_arr == arr).all(): 
    print("Yes, both the arrays are same") 
else: 
    print("No, both the arrays are not same") 

shape of arr:  (76377, 50, 6)
shape of load_original_arr:  (76377, 50, 6)
Yes, both the arrays are same


In [11]:
(total_array.shape)

(76377, 50, 6)

In [ ]:
"""
Load data contents
"""
print('start!')
import numpy as np
shape = (76377, 50, 6)
loaded_arr = np.loadtxt("numpy-window.txt") 
load_original_arr = loaded_arr.reshape( 
    loaded_arr.shape[0], loaded_arr.shape[1] // shape[2], shape[2]) 
load_original_arr.shape

In [ ]:
"""
Create gesture class labels according to index
1: buddha clap
2: crank left
3: crank right
4: knob left
5: knob right
6: pushback
7: swipe
"""
y_array = np.empty((0,1), int)
for i in range(1,8):
    label_array = np.full((10911,1), i)
    y_array = np.append(y_array, label_array , axis = 0)
y_array

In [ ]:
x_array = load_original_arr

# y_array.shape
x_array.shape

In [ ]:
"""
Split into train-test data
"""
from tqdm import tqdm
from sklearn.model_selection import train_test_split
train_ratio = 0.7
class_size = 10911 ## number of samples per class
x_train = np.empty((0,50,6))
y_train = np.empty((0,1), int)
x_test = np.empty((0,50,6))
y_test = np.empty((0,1), int)
for i in tqdm(range(7)):
    start = i*(class_size)
    stop =  (i+1)*(class_size)-1
    for index, (x,y) in enumerate(zip(x_array[start:stop], y_array[start:stop])):
        # print(index)
        if index < train_ratio*class_size:
            x_train = np.insert(x_train, 0, x, axis = 0)
            y_train = np.insert(y_train, 0, y, axis = 0)
        else:
            x_test = np.insert(x_test, 0, x, axis = 0)
            y_test = np.insert(y_test, 0, y, axis = 0)
x_train
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
"""
SAVE INTO text & numpy
"""
## save into numpy file
np.save('ready_data/x_train.npz', x_train)
np.save('ready_data/y_train.npz', y_train)
np.save('ready_data/x_test.npz', x_test)
np.save('ready_data/y_test.npz', y_test)



# reshaping the array from 3D 
# matrice to 2D matrice. 
arr_reshaped = x_train.reshape(x_train.shape[0], -1) 
arr_reshaped1 = x_test.reshape(x_test.shape[0], -1)
arr_reshaped2 = y_train#.reshape(y_train.shape[0], -1)
arr_reshaped3 = y_test#.reshape(y_test.shape[0], -1)

# saving reshaped array to file. 
np.savetxt("ready_data/x_train.txt", arr_reshaped) 
np.savetxt("ready_data/x_test.txt", arr_reshaped1) 
np.savetxt("ready_data/y_train.txt", arr_reshaped2) 
np.savetxt("ready_data/y_test.txt", arr_reshaped3) 